In [1]:
import plotly
import plotly.graph_objs as go
import numpy as np
import json
from common.h36m_dataset import Human36mDataset
%matplotlib inline
plotly.offline.init_notebook_mode(connected=True)

In [2]:
human36m_path = "./data/data_3d_h36m.npz"
human36m = Human36mDataset(human36m_path)
skeleton = human36m.skeleton()

In [3]:
human36m_kpts_name = ['Pelvis', 'RHip', 'RKnee', 'RAnkle','LHip','LKnee',
                      'LAnkle','Spine1','Neck', 'Head','Site','LShoulder',
                      'LElbow','LWrist','RShoulder', 'RElbow','RWrist']

In [4]:
pts_3d = np.load("./wild_data/yalu/box.npy")

In [5]:
pts_3d = pts_3d[:,:,[0,2,1]]
pts_3d = pts_3d[:,:,[1,0,2]]

In [6]:
obj_pts = pts_3d.reshape(-1,3)
xmin = obj_pts[:,0].min()
xmax = obj_pts[:,0].max()
ymin = obj_pts[:,1].min()
ymax = obj_pts[:,1].max()
zmin = obj_pts[:,2].min()
zmax = obj_pts[:,2].max()

print("xmin, xmax", xmin, xmax)
print("ymin, ymax", ymin, ymax)

xmin, xmax -11.459874893771065 19.646576196387734
ymin, ymax -13.368016642301232 12.986789133967875


In [7]:
def get_line_3d_segs(pts, k, skeleton, human36m_kpts_name, traj_p = 0):
    line_3d_segs = []
    cols = []
    for j, j_parent in enumerate(skeleton.parents()):
        if j_parent == -1:
            continue

        col = 'red' if j in skeleton.joints_right() else 'black'
        cols.append(col)
        trace = go.Scatter3d(
                    x = [pts[k, j, 0], pts[k, j_parent, 0]], 
                    y = [pts[k, j, 1], pts[k, j_parent, 1]], 
                    z = [pts[k, j, 2], pts[k, j_parent, 2]],
                    marker = dict(size=2, color="blue"),
                    text = [human36m_kpts_name[j]],
                    line = dict(color=col, width=5))
        line_3d_segs.append(trace)
    
    #if k != 0:
    trajectory = go.Scatter3d(
                    x = pts[:k, traj_p, 0], 
                    y = pts[:k, traj_p, 1], 
                    z = pts[:k, traj_p, 2],
                    marker = dict(color='#1f77b4', size=1),
                    line = dict(color='#1f77b4', width=1))
    
    surface = go.Mesh3d(
                    x = [-50, 50, -50, 50],
                    y = [-50, 50, 50, -50],
                    z = [0, 0, 0, 0],
                    color='#00FFFF',
                )
    
    line_3d_segs.append(trajectory)
    line_3d_segs.append(surface)
        
    return line_3d_segs

In [9]:
def generate_frames_layout(pts, skeleton, human36m_kpts_name, x_range = [-2, 5], y_range = [-2, 2], z_range = [-2, 5]):
    frames = []

    for k in range(len(pts)):
        frames.append(dict(data = get_line_3d_segs(pts, k, skeleton, human36m_kpts_name, traj_p = 3), name="frame{}".format(k+1)))    
    
    sliders=[
        dict(
            steps=[dict(method='animate',
                        args= [['frame{}'.format(k + 1)],
                                dict(mode='immediate',
                                     frame= dict(duration=70, redraw= True),
                                     transition=dict(duration=0))],
                                label='{:d}'.format(k+1)) for k in range(len(pts))], 
                        transition= dict(duration=0),
                    x=0,
                    y=0, 
                    currentvalue=dict(font=dict(size=12), 
                                      prefix='slice: ', 
                                      visible=True, 
                                      xanchor='center'),  
                    len=1.0)]
    
    # x_range = [-2, 5]
    # y_range = [-2, 2]
    # z_range = [-2, 5]
    t_sum = x_range[1]-x_range[0] + y_range[1]-y_range[0] + z_range[1]-z_range[0]
    ratio = ((x_range[1]-x_range[0])/t_sum, (y_range[1]-y_range[0])/t_sum, (z_range[1]-z_range[0])/ t_sum)
    ratio = np.array(ratio)*3
    layout = dict(
                width=800,
                height=700,
                autosize=False,
                title='action animation',
                scene=dict(
                    camera=dict(
                        up=dict(
                            x=0,
                            y=1,
                            z=0
                        ),
                        eye=dict(
                            x=-1.7428,
                            y=1.0707,
                            z=0.7100,
                        )
                    ),
                    xaxis=dict(
                        range=x_range),
                    yaxis=dict(
                        range=y_range),
                    zaxis=dict(
                        range=z_range),
                    aspectratio = dict( x=ratio[0], y=ratio[1], z=ratio[2] ),
                ),
                updatemenus=[
                    dict(type='buttons',
                         showactive=False,
                         y=1,
                         x=1.3,
                         xanchor='right',
                         yanchor='top',
                         pad=dict(t=0, r=10),
                         buttons=[dict(label='Play',
                                       method='animate',
                                       args=[None])])
                ],
                sliders=sliders
            )
    return frames, layout

In [10]:
frames, layout = generate_frames_layout(pts_3d[::100], 
                                        skeleton, 
                                        human36m_kpts_name, 
                                        x_range = [xmin, xmax], 
                                        y_range = [ymin, ymax], 
                                        z_range = [-1, zmax])

# frames, layout = generate_frames_layout(pts_3d[:5], 
#                                         skeleton, 
#                                         human36m_kpts_name, 
#                                         x_range = [-50, 50], 
#                                         y_range = [-50, 50], 
#                                         z_range = [-50, 50])

fig = dict(data=get_line_3d_segs(pts_3d, 0, skeleton, human36m_kpts_name), layout=layout, frames=frames)
plotly.offline.iplot(fig)

In [ ]:
pts_3d.shape